In [1]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=2, minor=7, micro=12, releaselevel='final', serial=0)
2.7.12 (default, Nov 19 2016, 06:48:10) 
[GCC 5.4.0 20160609]


In [5]:
# Import dataset
# Python 2
# pip install jieba
# pip install pyldavis

# -*- coding:utf-8 -*-

import pandas as pd
df = pd.read_csv("cleaned.csv")
# df = pd.read_csv("datascience.csv", encoding='gb18030')

df.head()

,content
0,老 用户 有 特权 恭喜 您 获得 光纤 宽带 不 加价 升 资格 年底 前 凭 机主 有效...
1,近期 有 市民 手机 接到 内容 涉及 含 同学聚会 揭发 隐私 等 内容 要求 点击 链接...
2,您 有 一份 流量 大礼 未 领取 亲 开通 功能 天内 可以 领取 超大 广州 本地 免费...
3,尊敬 的 客户 您 已 成功 开通 中国电信 总机 服务 业务 你 的 短 号码 为 感谢您...
4,平安 豪礼 提前 送 苹果 送到 手 欢 大礼 场 眼镜 各种 优惠券 等 你 扫走 快 戳...


In [6]:
df.shape

(171993, 1)

In [7]:
import jieba
def chinese_word_cut(mytext):
    return " ".join(jieba.cut(mytext))
# df["content_cutted"] = df#.apply(chinese_word_cut)

df["content_cutted"] = df.content.apply(chinese_word_cut)
df.content_cutted.head()

Building prefix dict from the default dictionary ...
Loading model from cache /tmp/jieba.cache
Loading model cost 0.353 seconds.
Prefix dict has been built succesfully.


0    老   用户   有   特权   恭喜   您   获得   光纤   宽带   不   ...
1    近期   有   市民   手机   接到   内容   涉及   含   同学聚会   揭...
2    您   有   一份   流量   大礼   未   领取   亲   开通   功能   ...
3    尊敬   的   客户   您   已   成功   开通   中国电信   总机   服务...
4    平安   豪礼   提前   送   苹果   送到   手   欢   大礼   场   ...
Name: content_cutted, dtype: object

In [8]:
df.content_cutted.shape

(171993,)

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
n_features = 1000
tf_vectorizer = CountVectorizer(strip_accents='unicode',
                                max_features=n_features,
                                stop_words='english',
                                max_df=0.5,
                                min_df=10)
tf = tf_vectorizer.fit_transform(df.content_cutted)

In [10]:
from sklearn.decomposition import LatentDirichletAllocation
n_topics = 5
# n_topics = 5
lda = LatentDirichletAllocation(n_components=n_topics,
                                max_iter=50,
                                learning_method='online',
                                learning_offset=50,
                                random_state=0
                               )
lda.fit(tf)

LatentDirichletAllocation(batch_size=128, doc_topic_prior=None,
             evaluate_every=-1, learning_decay=0.7,
             learning_method='online', learning_offset=50,
             max_doc_update_iter=100, max_iter=50, mean_change_tol=0.001,
             n_components=5, n_jobs=1, n_topics=None, perp_tol=0.1,
             random_state=0, topic_word_prior=None,
             total_samples=1000000.0, verbose=0)

In [11]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [12]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()
print_top_words(lda, tf_feature_names, n_top_words)

Topic #0:
回复 邮箱 账单 移动 通知 邮件 查看 中国移动 关闭 客服 提醒 资费 标准 计算 话费 满意 阅读 直接 送达 短信
Topic #1:
家长 孩子 老师 作业 小学 今天 完成 教育 各位 数学 学生 学校 明天 语文 通知 您好 学习 时间 下午 一个
Topic #2:
退订 回复 点击 流量 详情 积分 会员 活动 滴滴 免费 参与 关注 尊敬 领取 使用 即可 红包 优惠 电子 获得
Topic #3:
验证码 快递 尊敬 分钟 成功 短信 微信 手机 登录 账户 支付 联系 订单 操作 密码 尾号 使用 注册 余额 银行
Topic #4:
我们 退订 服务 尊敬 感谢您 您好 支持 工作 平安 祝您 出行 提示 电话 中国 温馨 客户 生活 时间 公司 预约
()


In [13]:
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()
pyLDAvis.sklearn.prepare(lda, tf, tf_vectorizer)

/SSD2/jack/pythonEnv/python2/local/lib/python2.7/site-packages/pyLDAvis/_prepare.py:387: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  topic_term_dists = topic_term_dists.ix[topic_order]


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
3      28.180229        1       1  0.072127 -0.171033
2      23.947470        1       2  0.148807 -0.153704
1      16.756510        1       3 -0.315344  0.194684
0      15.566451        1       4  0.259541  0.278042
4      15.549341        1       5 -0.165130 -0.147990, topic_info=     Category          Freq  Term         Total  loglift  logprob
term                                                             
925   Default  46080.000000    退订  46080.000000  30.0000  30.0000
313   Default  36567.000000    回复  36567.000000  29.0000  29.0000
393   Default  19635.000000    家长  19635.000000  28.0000  28.0000
940   Default  18302.000000    邮箱  18302.000000  27.0000  27.0000
998   Default  26752.000000   验证码  26752.000000  26.0000  26.0000
879   Default  16711.000000    账单  16711.000000  25.0000  25.0000
367   Default  15839.000000    孩子  15839.000000  24.0000  24.0000
758   Default  13602.000000    移动  13602.000000  23.0000  23.0000
676   Default  16586.000000    点击  16586.000000  22.0000  22.0000
938   Default  12429.000000    邮件  12429.000000  21.0000  21.0000
930   Default  17663.000000    通知  17663.000000  20.0000  20.0000
54    Default  11331.000000  中国移动  11331.000000  19.0000  19.0000
659   Default  20074.000000    流量  20074.000000  18.0000  18.0000
186   Default  10143.000000    关闭  10143.000000  17.0000  17.0000
803   Default  10363.000000    老师  10363.000000  16.0000  16.0000
388   Default   9313.000000    客服   9313.000000  15.0000  15.0000
485   Default   9194.000000    我们   9194.000000  14.0000  14.0000
618   Default  16478.000000    查看  16478.000000  13.0000  13.0000
859   Default  11371.000000    详情  11371.000000  12.0000  12.0000
890   Default   8428.000000    资费   8428.000000  11.0000  11.0000
622   Default   8382.000000    标准   8382.000000  10.0000  10.0000
132   Default   8389.000000    作业   8389.000000   9.0000   9.0000
840   Default   7871.000000    计算   7871.000000   8.0000   8.0000
529   Default  11705.000000    提醒  11705.000000   7.0000   7.0000
757   Default   9452.000000    积分   9452.000000   6.0000   6.0000
672   Default   7207.000000    满意   7207.000000   5.0000   5.0000
858   Default   8323.000000    话费   8323.000000   4.0000   4.0000
468   Default  10233.000000    快递  10233.000000   3.0000   3.0000
736   Default   6850.000000    直接   6850.000000   2.0000   2.0000
926   Default   6726.000000    送达   6726.000000   1.0000   1.0000
...       ...           ...   ...           ...      ...      ...
18     Topic5   2193.897838    上午   2194.813749   1.8607  -4.9746
93     Topic5   2024.731783    人满   2025.579557   1.8607  -5.0548
180    Topic5   2019.847516    公布   2020.696034   1.8607  -5.0572
339    Topic5   2026.430015    大家   2027.299218   1.8607  -5.0540
157    Topic5   1886.587421    做好   1887.464489   1.8607  -5.1255
158    Topic5   1790.808198    健康   1791.663844   1.8607  -5.1776
460    Topic5   1769.215947    影响   1770.072994   1.8607  -5.1897
226    Topic5   1616.820044    动态   1617.688954   1.8606  -5.2798
119    Topic5   1538.236159    企业   1539.089535   1.8606  -5.3296
135    Topic5   1555.176327    你好   1556.056911   1.8606  -5.3187
344    Topic5   1498.275070    天气   1499.135120   1.8606  -5.3559
497    Topic5   1485.361999    打开   1486.246051   1.8606  -5.3646
258    Topic5   1402.297207    发布   1403.153470   1.8605  -5.4221
368    Topic5   1352.249953    宁波   1353.100309   1.8605  -5.4585
668    Topic5   3646.104460    温馨   3863.736887   1.8032  -4.4666
528    Topic5   3752.855745    提示   4328.151343   1.7185  -4.4377
479    Topic5   4802.737736   感谢您   6711.428902   1.5265  -4.1911
599    Topic5   6611.291508    服务  12569.694147   1.2186  -3.8715
177    Topic5   3059.038428    公司   4182.014966   1.5485  -4.6422
679    Topic5   1798.466220    热线   1971.469316   1.7693  -5.1733
303    Topic5   2744.025230    咨询   4093.769448   1.4611  -4.7508
925    Topic5  